In [21]:
import numpy
from matplotlib import pyplot
from helper import l2_norm, poisson_solution, poisson_2d_jacobi

In [22]:
nx = 101
ny = 101
xmin, xmax = 0.0, 1.0
ymin, ymax = -0.5, 0.5
Lx = xmax - xmin
Ly = ymax - ymin
dx = Lx / (nx - 1)
dy = Ly / (ny - 1)  

x = numpy.linspace(xmin, xmax, num=nx)
y = numpy.linspace(ymin, ymax, num=nx)
X, Y = numpy.meshgrid(x, y)

b = (-2 * (numpy.pi / 2)**2 *
     numpy.sin(numpy.pi * X / Lx) *
     numpy.cos(numpy.pi * Y / Ly))

p0 = numpy.zeros((ny, nx))

pe = poisson_solution(x, y, Lx, Ly)

In [28]:
def poisson_2d_steepest_descent(p0, b, dx, dy, maxiter=20000, rtol=1e-6):
    def A(p):
        return ((p[1:-1, :-2] - 2 * p[1:-1, 1:-1] + p[1:-1, 2:]) / dx**2 +
                (p[:-2, 1:-1] - 2 * p[1:-1, 1:-1] + p[2:, 1:-1]) / dy**2)
    p = p0.copy()
    diff = rtol + 1
    ite = 0
    rk = numpy.zeros_like(p)
    Ar = numpy.zeros_like(p)
    while diff > rtol and ite < maxiter:
        pk = p.copy()
        rk[1:-1, 1:-1] = b[1:-1, 1:-1] - A(pk)
        Ar[1:-1, 1:-1] = A(rk)
        alpha = numpy.sum(rk * rk) / numpy.sum(rk * Ar)
        p = pk + alpha * rk
        diff = l2_norm(p, pk)
        ite += 1
    return p, ite, diff

In [29]:
p, ites, diff = poisson_2d_steepest_descent(p0, b, dx, dy, maxiter=20000, rtol=1e-10)
print(ites, diff)

2 1.2374874917997386e-16


In [33]:
def poisson_2d_cg(p0, b, dx, dy, maxiter=20000, rtol=1e-6):
    def A(p):
        return ((p[1:-1, :-2] - 2 * p[1:-1, 1:-1] + p[1:-1, 2:]) / dx**2 +
                (p[:-2, 1:-1] - 2 * p[1:-1, 1:-1] + p[2:, 1:-1]) / dy**2)
    p = p0.copy()
    r = numpy.zeros_like(p)  
    Ad = numpy.zeros_like(p)  
    diff = rtol + 1  
    ite = 0 
    r[1:-1, 1:-1] = b[1:-1, 1:-1] - A(p)
    d = r.copy()
    while diff > rtol and ite < maxiter:
        pk = p.copy()
        rk = r.copy()
        Ad[1:-1, 1:-1] = A(d)
        alpha = numpy.sum(r * r) / numpy.sum(d * Ad)
        p = pk + alpha * d
        r = rk - alpha * Ad
        beta = numpy.sum(r * r) / numpy.sum(rk * rk)
        d = r + beta * d
        diff = l2_norm(p, pk)
        ite += 1
    return p, ite, diff

In [34]:
p, ites, diff = poisson_2d_cg(p0, b, dx, dy, maxiter=20000, rtol=1e-10)
print(ites, diff)

2 1.2168165545926162e-16


In [38]:
p, ites, diff = poisson_2d_jacobi(p0, b, dx, dy, maxiter=40000, rtol=1e-10)
print('Jacobi relaxation: {} iterations '.format(ites) +
      'to reach a relative difference of {}'.format(conv_jacobi[-1]))

Jacobi relaxation: 31226 iterations to reach a relative difference of 9.997923503623598e-11


In [35]:
b = (numpy.sin(numpy.pi * X / Lx) *
     numpy.cos(numpy.pi * Y / Ly) +
     numpy.sin(6 * numpy.pi * X / Lx) *
     numpy.sin(6 * numpy.pi * Y / Ly))

In [36]:
maxiter, rtol = 40000, 1e-10
p, ites, conv = poisson_2d_jacobi(p0, b, dx, dy, maxiter=maxiter, rtol=rtol)
print('Jacobi relaxation: {} iterations'.format(ites))
p, ites, conv = poisson_2d_steepest_descent(p0, b, dx, dy, maxiter=maxiter, rtol=rtol)
print('Method of steepest descent: {} iterations'.format(ites))
p, ites, conv = poisson_2d_conjugate_gradient(p0, b, dx, dy, maxiter=maxiter, rtol=rtol)
print('Method of conjugate gradients: {} iterations'.format(ites))

Jacobi relaxation: 31226 iterations
Method of steepest descent: 26295 iterations
Method of conjugate gradients: 3 iterations
